In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [4]:
print(torch.cuda.get_device_name(0))  # Device number from above
print(torch.cuda.memory_allocated()) # Returned the current memory usage in Bytes
print(torch.cuda.memory_cached())

GeForce GTX 1050 Ti
0
0


In [5]:
a = torch.FloatTensor([1.0, 2.0])
a.device # Still used CPU

device(type='cpu')

# <font color = red> Need to send all tensors and models to GPU manually by calling .cuda() at the end </font>

In [6]:
# Whenever you create a tensor, you need to use CUDA along with it.
a = torch.FloatTensor([1.0, 2.0]).cuda()
print(a.device) # Now uses GPU
print(torch.cuda.memory_allocated()) 
print(torch.cuda.memory_cached())

cuda:0
512
2097152


In [7]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [8]:
torch.manual_seed(32)
model = Model()

In [9]:
next(model.parameters()).is_cuda # Model also has to be send to CUDA

False

## <font color = red> Send Model to GPU </font>

In [10]:
gpumodel = model.cuda()
next(gpumodel.parameters()).is_cuda # Need to send model to GPU as well.

True

In [11]:
df = pd.read_csv('../PYTORCH_NOTEBOOKS/Data/iris.csv')
X = df.drop('target',axis=1).values
y = df['target'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

## <font color = red> Convert Tensors to .cuda() tensors </font>

In [12]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

## <font color = red> Need to make pin_memory to True for GPU </font>

In [13]:
trainloader = DataLoader(X_train, batch_size=60, shuffle=True, pin_memory=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False, pin_memory=True)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [15]:
# Note DataLoader is not used here since it is a small dataset

import time
epochs = 100
losses = []
start = time.time()
for i in range(epochs):
    i+=1
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%10 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'TOTAL TRAINING TIME: {time.time()-start}')

epoch:  1  loss: 1.15071142
epoch: 11  loss: 0.93773186
epoch: 21  loss: 0.77982736
epoch: 31  loss: 0.60996711
epoch: 41  loss: 0.40083539
epoch: 51  loss: 0.25436994
epoch: 61  loss: 0.15052448
epoch: 71  loss: 0.10086147
epoch: 81  loss: 0.08127660
epoch: 91  loss: 0.07230931
TOTAL TRAINING TIME: 0.8776543140411377
